In [1]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf

In [2]:
database = pd.read_csv("../../Database/dataBaseWithNER.csv")

database = database.drop(columns=["Unnamed: 0"])
database = database.dropna()
target = database["target"].array
database

,email,target
0,start date hourahead timee cardinall hou...,0
1,service long desk price structure deal quote ...,0
2,start date cardinall hourahead timee card...,0
3,start date hourahead timee cardinall anc...,0
4,cardinall deliverable revenue management marke...,0
...,...,...
33340,bio matrix scientific group symbo bmxg p...,1
33341,cardinall step away hot naked webcam girl liv...,1
33342,need pill increase performance click seroius ...,1
33343,datee final nom inlet hpl eastrans car...,0


In [3]:
emailsText = []
for email in database["email"]:
    emailsText.append(email)

In [4]:
vectorizer = CountVectorizer(max_features=2100)
XTrain = vectorizer.fit_transform(emailsText)

bag = pd.DataFrame(XTrain.toarray(),columns=vectorizer.get_feature_names())

bag

E:\DevPack\anaconda3\envs\data_science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,ability,able,absolutely,abuse,accept,acceptance,accepted,access,according,...,xanax,xl,xp,yahoo,year,yes,yield,yo,young,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33336,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
33337,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
33338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_treino, X_teste, y_treino, y_teste = train_test_split(bag.values,target,test_size=0.2)

In [6]:
# Merge inputs and targets
inputs = np.concatenate((X_treino, X_teste), axis=0)
targets = np.concatenate((y_treino, y_teste), axis=0)

In [7]:
acc_per_fold = []
loss_per_fold = []

In [8]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

In [9]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

    # Define the model architecture
    model = keras.models.Sequential([
        ########## MLP
        keras.layers.Flatten(input_shape=(X_treino.shape[1],)),
        #keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(1000, activation="relu"),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),

        keras.layers.Dense(len(set(y_treino)), activation="softmax")
    ])

    # Compile the model
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd",
                  metrics=['accuracy'])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

    # Fit data to model
    history = model.fit(inputs[train], targets[train], epochs=200, callbacks=callbacks, validation_split=0.05)

    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/200
792/792 [==============================] - 3s 2ms/step - loss: 0.4425 - accuracy: 0.8525 - val_loss: 0.2757 - val_accuracy: 0.8958 - lr: 0.0100
Epoch 2/200
792/792 [==============================] - 2s 2ms/step - loss: 0.2424 - accuracy: 0.9295 - val_loss: 0.1358 - val_accuracy: 0.9543 - lr: 0.0100
Epoch 3/200
792/792 [==============================] - 2s 2ms/step - loss: 0.1289 - accuracy: 0.9592 - val_loss: 0.1100 - val_accuracy: 0.9685 - lr: 0.0100
Epoch 4/200
792/792 [==============================] - 2s 2ms/step - loss: 0.2024 - accuracy: 0.9537 - val_loss: 0.1600 - val_accuracy: 0.9393 - lr: 0.0100
Epoch 5/200
792/792 [==============================] - 2s 2ms/step - loss: 0.2703 - accuracy: 0.9555 - val_loss: 0.1524 - val_accuracy: 0.9573 - lr: 0.0100
Epoch 6/200
792/792 [==============================] - 2s 2ms/step - loss: 0.1154 - accuracy: 0.9701 - val_loss: 0.1125 - v

In [10]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.08282823115587234 - Accuracy: 97.27095365524292%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.09300064295530319 - Accuracy: 97.28554487228394%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.07573442161083221 - Accuracy: 97.51049876213074%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.07687965035438538 - Accuracy: 97.57048487663269%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.07118797302246094 - Accuracy: 98.17036390304565%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 97.56156921386719 (+- 0.3267921338453489)
> Loss: 0.07992618381977082
----------------------